# Latencia
Tiempo que tarda un modelo en procesar una entrada y generar una prediccion

In [1]:
import os, time
import importlib , latency, pandas as pd # Importar importlib para recargar módulos
from dotenv import load_dotenv

dotenv_path = os.path.join(os.getcwd(), '..', '.env')
importlib.reload(latency)
from glob import glob
from latency import Latency

# Configuracion
config_file = os.path.join(os.getenv('BASE_PATH'), os.getenv('MMPOSE_SUBPATH'), os.getenv('CONFIG_SUBPATH'))
checkpoint_file = os.path.join(os.getenv('BASE_PATH'), os.getenv('MMPOSE_SUBPATH'), os.getenv('CHECKPOINT_SUBPATH'))
images_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('IMAGES_SUBPATH'))
labels_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('LABELS_SUBPATH'))
videos_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('VIDEOS_SUBPATH'))
evaluator = Latency(os.getenv('BASE_PATH'), images_path, labels_path, videos_path, config_file, checkpoint_file)
thresholds = [0.1, 0.5, 0.9]

/Users/apple/Documents/pose_estimation/MMPosev1.1/mmpose_env/lib/python3.10/site-packages/mmcv/cnn/bricks/transformer.py:33: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv`` rather than ``mmcv-lite`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '


Loads checkpoint by local backend from path: /Users/apple/Documents/pose_estimation/MMPosev1.1/configs/model_weights/hrnet_w32_coco_256x192-c78dce93_20200708.pth


/Users/apple/Documents/pose_estimation/MMPosev1.1/mmpose_env/lib/python3.10/site-packages/mmpose/datasets/datasets/utils.py:102: UserWarning: The metainfo config file "configs/_base_/datasets/coco.py" does not exist. A matched config file "/Users/apple/Documents/pose_estimation/MMPosev1.1/mmpose_env/lib/python3.10/site-packages/mmpose/.mim/configs/_base_/datasets/coco.py" will be used instead.
  warnings.warn(


# Calculo de latencia en un video especifico

In [ ]:
# Obtener la lista de todas las imágenes en la carpeta
video = 'ejercicio01 - frente.mp4'
video_path = os.path.join(videos_path, 'uziel', 'ejercicio01', 'conLuz', video)
print(video_path)

results = []
for threshold in thresholds:
    results = evaluator.evaluate_video(
        videos_path=video_path, 
        threshold=threshold,
        results=results,
        frame_skip=5,
    )

# Calculo de latencia en videos dentro de una carpeta

In [2]:

# folder_path = os.path.join(videos_path, 'uziel')
folder_path = os.path.join(videos_path)
video_extensions = ['*.mp4', '*.avi', '*.mov']
video_files = []
results = []

for ext in video_extensions:
    video_files.extend(glob(os.path.join(folder_path, '**', ext), recursive=True))

for video_path in video_files:
    for threshold in thresholds:
        results = evaluator.evaluate_video(
            videos_path=video_path, 
            threshold=threshold, 
            results=results,
            frame_skip=5,
        )

Procesando (/Users/apple/Documents/pose_estimation/dataset/videos_resized/persona5/ejercicio02/conLuz/ejercicio02 - frente.mp4) - (th:0.1): 100%|██████████| 513/513 [00:24<00:00, 21.06it/s]
Procesando (/Users/apple/Documents/pose_estimation/dataset/videos_resized/persona5/ejercicio02/conLuz/ejercicio02 - frente.mp4) - (th:0.5): 100%|██████████| 513/513 [00:24<00:00, 20.73it/s]
Procesando (/Users/apple/Documents/pose_estimation/dataset/videos_resized/persona5/ejercicio02/conLuz/ejercicio02 - frente.mp4) - (th:0.9): 100%|██████████| 513/513 [00:24<00:00, 20.93it/s]
Procesando (/Users/apple/Documents/pose_estimation/dataset/videos_resized/persona5/ejercicio02/conLuz/ejercicio02 - derecha.mp4) - (th:0.1): 100%|██████████| 466/466 [00:22<00:00, 20.98it/s]
Procesando (/Users/apple/Documents/pose_estimation/dataset/videos_resized/persona5/ejercicio02/conLuz/ejercicio02 - derecha.mp4) - (th:0.5): 100%|██████████| 466/466 [00:22<00:00, 20.91it/s]
Procesando (/Users/apple/Documents/pose_estimati

# Generando XLS de los resultados de los videos

In [3]:
df_results = pd.DataFrame(results)
print(df_results.head().to_string)
output_excel_path = os.path.join(os.getenv('BASE_PATH'), os.getenv('MMPOSE_SUBPATH'), 'velocidad', 'latency', 'mmpose_video_latency_results.xlsx')
df_results.to_excel(output_excel_path, index=False, engine='openpyxl')

print(f"Los resultados se han guardado en {output_excel_path}")

<bound method DataFrame.to_string of                       video video_resolution  threshold  video_fps  \
0  ejercicio02 - frente.mp4          144x256        0.1      29.96   
1  ejercicio02 - frente.mp4          144x256        0.1      29.96   
2  ejercicio02 - frente.mp4          144x256        0.1      29.96   
3  ejercicio02 - frente.mp4          144x256        0.1      29.96   
4  ejercicio02 - frente.mp4          144x256        0.1      29.96   

   frame_skip  latency_ms  processed_frames  total_frames  
0           5  355.360985                 1           513  
1           5  277.167797                 2           513  
2           5  234.771013                 3           513  
3           5  232.608080                 4           513  
4           5  235.919952                 5           513  >
Los resultados se han guardado en /Users/apple/Documents/pose_estimation/MMPosev1.1/velocidad/latency/mmpose_video_latency_results.xlsx
